## 크롤러 환경설정

In [69]:
import re
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas as pd


options = webdriver.ChromeOptions()
options.add_argument("headless")

각자 chromedriver.exe 경로 수정

In [70]:
driver_path = 'C:/Users/chaeh/chromedriver.exe'

특수문자 및 영어등 텍스트 전처리 함수 -> clean_text

In [71]:
def clean_text(text):
    cleaned_text = re.sub('[a-zA-Z]', '', text)
    cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]',
                          '', cleaned_text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

## 경향신문

In [72]:
def biz_khan(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 
    last_page =(int(page)-1)*10+1
    domain = '경향신문'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options) 
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1032&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
            driver.get(url)
            time.sleep(2)
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup.find('h1',id='articleTtitle')
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.find('h1',id='articleTtitle').text)
                tags = soup.select_one('#container > div.main_container > div.art_cont > div.art_body').find_all('p')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                try:
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    head_list.append(soup.find('h1',id='article_title').text)
                    tags = soup.select_one('#articleBody').find_all('p')
                    data = [clean_text(x.text) for x in tags]
                    data_list.append(' '.join(data))
                except:
                    print(url)
                    print('None_type')
            driver.close()
        current_page+=10

    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 국민일보

In [73]:
def kmib(query,driver_path,page):
    domain = '국민일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    data_list = []
    head_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1005&nso=so%3Ar%2Cp%3Aall&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        #print(url)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()
        
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
            time.sleep(2)
            driver.get(url)
            time.sleep(2)
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select_one('#sub > div.sub_header > div > div.nwsti > h3').text)
                tags = soup.select_one("#articleBody").text
                data=tags.strip()
                data=data.strip('\t')
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
             
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 내일신문

In [74]:
def naeil(query,driver_path,page):
    domain = '내일신문'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    data_list = []
    head_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2312&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        #print(url)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
            time.sleep(2)
            driver.get(url)
            time.sleep(2)
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select_one('#contentArea > div.caL2 > div > div.articleArea > h3').text)
                tags = soup.select_one("#contents").text
                data=tags.strip()
                data=data.strip('\t')
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10  
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 동아일보

In [75]:
def donga(query,driver_path, page):
    domain = '동아일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    
    current_page =1
    last_page =(int(page)-1)*10+1
    
    data_list =[]
    head_list=[]
    
    while current_page < last_page:
        driver =webdriver.Chrome(executable_path =driver_path, options =options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1020&nso=&is_sug_officeid=0'
        url = url_domain + query + '&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
 
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
    
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
            
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()
    
    
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path,options=options)
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
        # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select_one('#container > div.article_title > h1').text)
                tags = soup.select_one('#content > div > div.article_txt').text
                tags=clean_text(tags)
                data_list.append(tags)
            except:
                try:
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    head_list.append(soup.select_one('#content > div.article_title > h2').text)
                    tags = soup.select_one('#ct').text
                    tags=clean_text(tags)
                    data_list.append(tags)
                except:
                    print(url)
                    print('None_type')
            driver.close()
        current_page += 10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 매일일보

In [76]:
def m_i(query,driver_path,page):
    domain = '매일일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options) 
    
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    data_list = []
    head_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path =driver_path, options=options)
        url_domain ='https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        print(url)
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
    
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
    
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
    
        except: 
            print('기사가 없습니다.')
            driver.close()
            return ()
        driver.close()
    
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path,options=options) 
            
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
        # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select('#user-container > div.custom-pc.float-center.max-width-1130 > header > div > div')[0].text)
                tags = soup.find(id='article-view-content-div').find_all('p')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page += 10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 문화일보

In [77]:
def munhwa(query,driver_path,page):
    domain = '문화일보'
    
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path)
        driver = webdriver.Chrome(executable_path = driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1021&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
                
        except:
            print('기사가 없습니다')
            driver.close()
            return 0
        driver.close()
        
        
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
            try:
                driver.get(url)
                time.sleep(1)
            
            except:
                print('link error')
                continue
                # try문 안에서 각자 언론사에 맞게 수정
                
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.find('span', class_ = 'title').text)
                data = soup.select('#NewsAdContent')[0].text
                data=data.strip()
                data=data.strip('\t')
                data=clean_text(data)
                data_list.append(data)
            
            except:
                try:
                    soup = BeautifulSoup(driver.page_source,'html.parser')
                    head_list.append(soup.find('h3', id = 'articleTitle').text)
                    data = soup.select('#articleBodyContents')[0].text
                    data=data.strip()
                    data=data.strip('\t')
                    data=clean_text(data)
                    data_list.append(data)
                except:
                    print(url)
                    print("None_type")
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 서울신문

In [78]:
def seoul(query,driver_path,page):
    domain = '서울신문'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    data_list = []
    head_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1081&nso=&is_sug_officeid=0' 
        url = url_domain + query + '&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()
        
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path,options=options)  # for Windows
            time.sleep(2)
            driver.get(url)
            time.sleep(2)
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select_one('#viewWrapDiv > div.S20_title > div.S20_article_tit > h1').text)
                tags = soup.select_one("#atic_txt1").text
                data=tags.strip()
                data=data.strip('\t')
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
             
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 세계일보

In [81]:
def segye(query,driver_path,page):
    domain = '세계일보'
    driver = webdriver.Chrome(executable_path=driver_path)
        
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1022&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page)+ news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()


        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path) 
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
                
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select('#title_sns')[0].text)
                data = soup.find('article',class_='viewBox').text
                data=data.strip()
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 아시아투데이

In [82]:
def asiatoday(query,driver_path,page):
    domain = '아시아투데이'
    driver = webdriver.Chrome(executable_path=driver_path)
    
    current_page = 1 
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2268&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain

        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > sec6tion > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()


        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
            try:
                driver.get(url)
                time.sleep(1)
            except:
                print('link error')
                continue
            # try문 안에서 각자 언론사에 맞게 수정
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(soup.select('#section_top > div > h3')[0].text)
                data = soup.find('div',class_='news_bm').text
                data=data.strip()
                data=clean_text(data)
                data_list.append(data)
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 전국매일신문

In [83]:
def alldays(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 
    last_page =(int(page)-1)*10+1
    domain = '전국매일신문'
    
    while current_page < last_page:       
        driver = webdriver.Chrome(executable_path=driver_path, options=options) 
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2844&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find('div',class_="article-head-title").text)
                head_list.append(head)
                tags = soup.select_one('#article-view-content-div').find_all('p')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print(url)
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 조선일보

In [84]:
def joseon(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1
    last_page =(int(page)-1)*10+1
    domain = '조선일보'
    
    while current_page < last_page:        
        driver = webdriver.Chrome(executable_path=driver_path, options=options)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1023&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find_all("h1", class_="article-header__headline | font--primary text--black")[0].text)
                head_list.append(head)
                tags = soup.find_all('p',class_=' article-body__content article-body__content-text | text--black text font--size-sm-18 font--size-md-18 font--primary')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 중앙일보

In [85]:
def chungang(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 
    last_page =(int(page)-1)*10+1
    domain = '중앙일보'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options)  
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1025&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.select('#article_title')[0].text)
                head_list.append(head)
                text = soup.find('div', id='article_body').text
                text = clean_text(text)
                data_list.append(text)
            except:
                print(url)
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 천지일보

In [86]:
def newscj(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1
    last_page =(int(page)-1)*10+1
    domain = '천지일보'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options) 
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2041&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options) 
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find('div',class_="article-head-title").text)
                head_list.append(head)
                tags = soup.select_one('#article-view-content-div').find_all('p')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print(url)
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 한겨례

In [87]:
def hani(query,driver_path,page):
    head_list = []
    data_list = []
    current_page = 1 
    last_page =(int(page)-1)*10+1
    domain = '한겨례'
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path=driver_path, options=options) 
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain ='&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1028&nso=&is_sug_officeid=0' 
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')

        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
        except:
            print('기사가 없습니다.')
            driver.close()
            return 0
        driver.close()

        for url in url_list:
            try:
                driver = webdriver.Chrome(executable_path=driver_path, options=options)  
                driver.get(url)
                time.sleep(2)
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head = clean_text(soup.find('span', class_='title').text)
                head_list.append(head)
                tags = soup.select_one('#a-left-scroll-in > div.article-text > div').find_all('div', class_='text')
                data = [clean_text(x.text) for x in tags]
                data_list.append(' '.join(data))
            except:
                print("None_type")
            driver.close()
        current_page+=10
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                           '제목' :  head_list,
                           '내용' : data_list})
    return output

## 한국일보

In [160]:
def hankook(query,driver_path,page):
    domain = '한국일보'
    driver = webdriver.Chrome(executable_path=driver_path, options=options)  # for Windows
    
    current_page = 1
    last_page =(int(page)-1)*10+1
    
    head_list = []
    data_list = []
    
    
    while current_page < last_page:
        driver = webdriver.Chrome(executable_path = driver_path)
        url_domain = 'https://search.naver.com/search.naver?where=news&query='
        news_domain = '%EB%89%B4%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1469&nso=&is_sug_officeid=0'
        url = url_domain + query +'&%2Ca%3A&start='+str(current_page) + news_domain
        driver.get(url)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        
        # 기사 url을 list형태로 반환
        try:
            news_list = soup.select('#main_pack > section > div > div.group_news > ul')
            url_list = []
            news_list = news_list[0].find_all('li')
            
            for url in news_list:
                url_list.append(str(url.find_all('a')[0].get('data-url')))
                
        except:
            print('기사가 없')
            driver.close()
            return 0
        driver.close()
        
        
        for url in url_list:
            driver = webdriver.Chrome(executable_path=driver_path)  # for Windows
            try:
                driver.get(url)
                time.sleep(1)
            
            except:
                print('link error')
                continue
                # try문 안에서 각자 언론사에 맞게 수정
                
            try:
                soup = BeautifulSoup(driver.page_source,'html.parser')
                head_list.append(clean_text(soup.find('h2').text))
                tags = soup.select_one('body > div.wrap > div.container.end.end-uni > div.end-body > div > div.col-main.read').find_all('p')
                data = [x.text for x in tags]
                data_list.append(data)
            
            except:
                print(url)
                print('None_type')
            driver.close()
        current_page+=10
        
    output = pd.DataFrame({'언론사' : [domain]*len(head_list),
                          '제목' :  head_list,
                          '내용' : data_list})
    return output

## 원하는 keyword와 페이지 수 설정

In [89]:
query = '"풀무원" 바른먹거리'
max_page=10

In [90]:
biz_df = biz_khan(query,driver_path,max_page)

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=032&aid=0000069224
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=0000070370
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=0000070370
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=0000070370
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=0000070370
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=0000070370
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=0000070370
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=0000070370
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=032&aid=0000070370
None_type


In [91]:
kmib_df = kmib(query,driver_path,max_page)

http://news.kukinews.com/article/view.asp?arcid=0008618427&code=41171811&cp=nv
None_type
http://news.kukinews.com/article/view.asp?arcid=0008613476&code=41171811&cp=nv
None_type
http://news.kukinews.com/article/view.asp?arcid=0008568919&code=41171811&cp=nv
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=005&aid=0000166450
None_type
http://news.kukinews.com/article/view.asp?page=1&gCode=cul&arcid=0007343144&cp=nv
None_type
http://news.kmib.co.kr/article/view.asp?arcid=0008453804&code=41171811&cp=nv
None_type
http://www.kukinews.com/news2/article/view.asp?page=1&gCode=all&arcid=1237210098&cp=nv
None_type


In [92]:
naeil_df = naeil(query,driver_path,max_page)

https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=086&aid=0000048442
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=086&aid=0002128522
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=086&aid=0002161049
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=086&aid=0002168403
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=086&aid=0002152391
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=086&aid=0002078723
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=086&aid=0001957511
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=086&aid=0001938848
None_type


In [93]:
donga_df = donga(query,driver_path,max_page)

In [95]:
mi_df = m_i(query,driver_path,max_page)

https://search.naver.com/search.naver?where=news&query="풀무원" 바른먹거리&%2Ca%3A&start=1&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0
https://search.naver.com/search.naver?where=news&query="풀무원" 바른먹거리&%2Ca%3A&start=11&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0
https://search.naver.com/search.naver?where=news&query="풀무원" 바른먹거리&%2Ca%3A&start=21&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=2385&nso=so%3Ar%2Cp%3Aall%2Ca%3Aall&is_sug_officeid=0
https://search.naver.com/search.naver?where=news&query="풀무원" 바른먹거리&%2Ca%3A&start=31&sm=tab_opt&sort=0&photo=0&field=0&pd=0&ds=&de=&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_of

In [96]:
munhwa_df = munhwa(query,driver_path,max_page)

In [97]:
seoul_df = seoul(query,driver_path,max_page)

https://biz.seoul.co.kr/news/newsView.php?id=20210315500025&wlog_tag3=naver
None_type
https://www.seoul.co.kr/news/newsView.php?id=20200612014002&wlog_tag3=naver
None_type
http://nownews.seoul.co.kr/news/newsView.php?id=20180530601018&wlog_tag3=naver
None_type
http://nownews.seoul.co.kr/news/newsView.php?id=20180809601005&wlog_tag3=naver
None_type
http://nownews.seoul.co.kr/news/newsView.php?id=20160414601010
None_type
http://nownews.seoul.co.kr/news/newsView.php?id=20170612601013&wlog_tag3=naver
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=103&oid=081&aid=0000082014
None_type
http://nownews.seoul.co.kr/news/newsView.php?id=20100903603077
None_type


In [99]:
segye_df = segye(query,driver_path,max_page)

link error
http://www.economysegye.com/articles/view.html?aid=20130326001515&cid=7111000000000&OutUrl=naver
None_type
link error
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=022&aid=0000092234
None_type
https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=102&oid=022&aid=0000092149
None_type


In [100]:
asia_df = asiatoday(query,driver_path,max_page)

In [101]:
alldays_df = alldays(query, driver_path,max_page)

기사가 없습니다.


In [102]:
joseon_df = joseon(query, driver_path,max_page)

In [103]:
chungang_df = chungang(query, driver_path,max_page)

기사가 없습니다.


In [104]:
cj_df = newscj(query,driver_path,max_page)

In [105]:
hani_df = hani(query,driver_path,max_page)

None_type
None_type


In [161]:
hk_df = hankook(query,driver_path,max_page)

In [162]:
df_list=[biz_df, kmib_df, naeil_df, donga_df, mi_df, munhwa_df, seoul_df, segye_df, asia_df, alldays_df ,joseon_df, chungang_df, cj_df, hani_df, hk_df]

In [180]:
names=[]
for name in df_list:
    if type(name)!=int:
        names.append(name)

In [181]:
news_df=pd.concat(names)
news_df.drop_duplicates(['제목'],inplace=True,ignore_index=True)

In [182]:
news_df

,언론사,제목,내용
0,경향신문,‘바른 먹거리’ 풀무원 물류센터에서는 무슨 일이?,지난 16일 사회관계망서비스에서는 차밑에 들어가 있는 한 노동자의 사진이 화제를 모...
1,경향신문,"어린이 ‘바른 먹거리 캠페인’ 신청하세요… 풀무원, 사회공헌 교육 확대",풀무원이 어린이들의 바른 식생활을 위해 2010년부터 실시해온 ‘바른 먹거리 캠페인...
2,경향신문,"풀무원, 어린이 바른 먹거리 현장 체험교육",풀무원 홀딩스는 초등학교 여름방학을 맞아 충북 음성 풀무원 두부공장을 돌아보는 ‘풀...
3,경향신문,풀무원 ‘바른 먹을거리 교육’에 어린이 신청 늘어,어린이를 대상으로 한 올바른 먹을거리 교육이 학교와 학부모들로부터 관심을 모으고 있...
4,경향신문,"풀무원, 13년 만에 새 CI 발표... 글로벌 로하스 도약 목표",풀무원 새 풀무원이 2005년 이후 13년 만에 기업이미지를 교체했다 계열사 브랜드...
...,...,...,...
303,한겨례,“일 25씩 더 해 기부하면 세계 굶주림 사라져”,“세상의 아름다움 중에서 인간의 아름다움을 딛고 넘어설 만한 아름다움은 없다고 생각...
304,한국일보,냉면·짜장·떡볶이 제품에 트레이가 필요한가요,"[[쓰레기를 사지 않을 권리]<15>즉석조리식품 트레이, 기후위기와 쓰레기산에 신음..."
305,한국일보,풀무원 입사 1호 사원이 … 성장세 발목 잡던 해외 실적도 개선,"[\n원혜영 의원 부친 농장, 남승우 前대표가 기업화\n, \n‘바른 먹거리’로 年..."
306,한국일보,풀무원 13년 만에 기업이미지 바꿔… 계열사 브랜드도 통일,[풀무원이 2005년 이후 13년 만에 기업이미지(CI)를 교체하고 계열사 브랜드 ...
